 ----------------------------------------**Model_Conversion.ipynb**-------------------------------------------------------------------

Part of the Startup-Demos Project, under the MIT License<br>
See https://github.com/qualcomm/Startup-Demos/blob/main/LICENSE.txt for license information.<br>
Copyright (c) Qualcomm Technologies, Inc. and/or its subsidiaries.<br>
SPDX-License-Identifier: MIT License

--------------------------------------------------------------------------------------------------------------

ResNet50 Emotion Architecture Extractor
---------------------------------------
- Downloads the `run_webcam.ipynb` from Hugging Face (ElenaRyumina/face_emotion_recognition),
- Extracts the model architecture cell containing `class Bottleneck` and `def ResNet50`,
- Writes a clean Python module `resnet50_emo.py`.

In [ ]:
# Extract the ResNet50 emotion model architecture from a Hugging Face notebook into a standalone Python module.

import os
import textwrap
import nbformat
import torch  # noqa: F401
from huggingface_hub import hf_hub_download

# 1) Download the notebook from Hugging Face (version-aware & cached)
nb_path = hf_hub_download(
    repo_id="ElenaRyumina/face_emotion_recognition",
    filename="run_webcam.ipynb",
    repo_type="model",
    # Optional: pin a specific revision for reproducibility:
    # revision="f4944b0..."
)

# 2) Parse the notebook and locate the model-architecture code cell
nb = nbformat.read(nb_path, as_version=4)

model_cell_src = None
for cell in nb.cells:
    if cell.get("cell_type") == "code":
        src = cell.get("source", "")
        if ("class Bottleneck" in src) and ("def ResNet50" in src):
            model_cell_src = src
            break

if model_cell_src is None:
    raise RuntimeError("Model architecture cell not found; the notebook may have changed.")

# 3) Clean cell code: strip notebook magics (e.g., %, !) if any
model_code_clean = "\n".join(
    line for line in model_cell_src.splitlines()
    if not line.strip().startswith("%") and not line.strip().startswith("!")
)

# 4) Write the architecture to a Python module
module_path = "resnet50_emo.py"
header = textwrap.dedent(
    """
    import math
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    """
).strip() + "\n\n"

with open(module_path, "w", encoding="utf-8") as f:
    f.write(header)
    f.write(model_code_clean)

print(f"[INFO] Saved model architecture to: {os.path.abspath(module_path)}")

ResNet50 Emotion Model Loader
-----------------------------
- Imports the extracted ResNet50 architecture, builds the model, loads a checkpoint.
- Performs a quick sanity check in eval mode.
- Download and load FER_static_ResNet50_AffectNet.pt from Hugging Face using huggingface_hub.

In [ ]:
# Load the pretrained ResNet50 AffectNet emotion model weights into the standalone PyTorch architecture

import torch
from huggingface_hub import hf_hub_download
from collections import OrderedDict
from resnet50_emo import ResNet50

# Download into local cache; version-aware
ckpt_path = hf_hub_download(
    repo_id="ElenaRyumina/face_emotion_recognition",
    filename="FER_static_ResNet50_AffectNet.pt",
    repo_type="model",
    # revision="main"  # optionally pin a specific commit/tag for reproducibility
)

# Build and load
model = ResNet50(num_classes=7, channels=3)
state_dict = torch.load(ckpt_path, map_location="cpu")

if any(k.startswith("module.") for k in state_dict.keys()):
    state_dict = OrderedDict((k.replace("module.", ""), v) for k, v in state_dict.items())

missing_unexp = model.load_state_dict(state_dict, strict=True)
print("[INFO] load_state_dict (missing, unexpected):", missing_unexp)

model.eval()


In [ ]:
# This cell converts the PyTorch emotion recognition model into an ONNX file for use in other frameworks.

import torch
import os

# Output path: current working directory
onnx_path = os.path.join(os.getcwd(), "resnet50_emotion.onnx")

# Dummy input (NCHW float32, 224x224)
dummy = torch.randn(1, 3, 224, 224, dtype=torch.float32)

torch.onnx.export(
    model,                       # your nn.Module
    dummy,                       # example input
    onnx_path,                   # output file
    input_names=["image"],       # match AI Hub input_specs
    output_names=["logits"],     # convenient name
    opset_version=13,            # good default for pad/conv
    do_constant_folding=True,
    dynamic_axes={"image": {0: "batch"}, "logits": {0: "batch"}},
    training=torch.onnx.TrainingMode.EVAL,
    export_params=True
)

print(f"[INFO] ONNX model saved to: {onnx_path}")



Compile ONNX with AI Hub
-----------------------------------------------------
Submits a compile job to Dragonwing RB3 Gen 2 Vision Kit using the ONNX
model found in the current directory.


In [ ]:
# Submit and run an AI Hub compile job to build the ONNX ResNet50 emotion model for the RB3 Gen 2 target

import os
from pathlib import Path
import qai_hub as hub

# Device
DEVICE = hub.Device("Dragonwing RB3 Gen 2 Vision Kit")

# ONNX path from current working directory
onnx_path = Path(os.getcwd()) / "resnet50_emotion.onnx"  # ensure this file exists

# Create and submit the compile job with input specs and runtime/output option
compile_job_static = hub.submit_compile_job(
    model=str(onnx_path),
    device=DEVICE,
    name="EMO_AffectNet_ONNX_model",
    input_specs={"image": ((1, 3, 224, 224), "float32")},
    options="--target_runtime onnx --output_names logits",
)
# Block until the compile job finishes and retrieve the compiled target model artifact
compile_job_static.wait()
static_model = compile_job_static.get_target_model()
print("[INFO] Compile job completed. Target model path:", static_model)

Calibartion Data
-----------------------
- Use Option A when you have a real image dataset in your working directory
- Option B when you want synthetic calibration samples (no external data needed).

__Note__ 
- For best quantization accuracy, Option A (real image dataset) is recommended because real samples better represent the activation. 
- Option B (synthetic samples) is useful for quick bring-up or when real data is unavailable, but may lead to slightly lower accuracy compared to real calibration data.
- Choose any one option(optionA or optionB) for running the Calibartion

Option A: Image Dataset Calibration Samples
----------------------------
- Manual Download the Image Dataset
- Go to [FER2013 Kaggle page](https://www.kaggle.com/datasets/msambare/fer2013).
- Click Download and unzip the dataset into your project folder (e.g., .archive/test/).
- Inside, you’ll typically see train, test, and validation folders.

In [ ]:
# Build a balanced set of preprocessed AffectNet test images (excluding 'contempt') as calibration_data['image'] for quantization/evaluation

from pathlib import Path
import os, random, cv2, numpy as np
from PIL import Image
from collections import Counter

# Point to the test split in your local folder
AFFECTNET_DIR = Path(os.getcwd()) / "archive" / "test"
IMG_EXTS = ('.jpg', '.jpeg', '.png', '.bmp')
EXCLUDE_CLASS = "contempt"
PER_CLASS_LIMIT = 100
RNG_SEED = 42

BGR_MEAN = np.array([91.4953, 103.8827, 131.0912], dtype=np.float32)

def preprocess_caffe_bgr_from_pil(pil_img, size=(224, 224)):
    rgb = np.asarray(pil_img.convert('RGB'))
    bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    bgr = cv2.resize(bgr, size, interpolation=cv2.INTER_LINEAR).astype(np.float32)
    bgr -= BGR_MEAN
    x = np.transpose(bgr, (2, 0, 1))
    return np.expand_dims(x, 0)

# Build dict: class -> image paths
class_to_paths = {}
for emotion_folder in os.listdir(AFFECTNET_DIR):
    folder_path = AFFECTNET_DIR / emotion_folder
    if not folder_path.is_dir():
        continue
    if emotion_folder.lower() == EXCLUDE_CLASS.lower():
        print(f"[INFO] Skipping folder: {emotion_folder}")
        continue
    imgs = [str(folder_path / f) for f in os.listdir(folder_path) if f.lower().endswith(IMG_EXTS)]
    if imgs:
        class_to_paths[emotion_folder] = imgs

# Summary
total_avail = sum(len(v) for v in class_to_paths.values())
print(f"Found {total_avail} images across {len(class_to_paths)} folders (excluding '{EXCLUDE_CLASS}').")

# Balanced selection
rng = random.Random(RNG_SEED)
selected_paths = []
for cls, paths in class_to_paths.items():
    rng.shuffle(paths)
    take_count = min(PER_CLASS_LIMIT, len(paths))
    selected_paths.extend(paths[:take_count])

rng.shuffle(selected_paths)

# Preprocess
sample_inputs, bad = [], 0
for i, img_path in enumerate(selected_paths):
    try:
        pil_img = Image.open(img_path)
        arr = preprocess_caffe_bgr_from_pil(pil_img)
        sample_inputs.append(arr)
        if i < 8:
            print(f"Sample {i}: {os.path.basename(img_path)} shape={arr.shape}")
    except Exception as e:
        bad += 1
        print(f"[Skip] {img_path}: {e}")

print(f"Prepared {len(sample_inputs)} calibration samples. Skipped {bad} problematic images.")

# Validate
expected_shape = (1, 3, 224, 224)
if not all(arr.shape == expected_shape for arr in sample_inputs):
    raise ValueError("Shape mismatch")
calibration_data = {"image": sample_inputs}
print(f"[OK] calibration_data['image'] has {len(calibration_data['image'])} samples.")


Option B: Synthetic Calibration Samples (No External Data Needed)
--------------------------
This approach generates artificial images that mimic natural image statistics for post-training quantization calibration

In [ ]:
## Synthetic calibration samples (no external data needed)

import numpy as np
import cv2
from PIL import Image

# --------- Configuration ---------
NUM_SAMPLES = 512
IMG_SIZE = (224, 224)              # (width, height)
INPUT_NAME = "image"
RNG_SEED = 42

# Caffe BGR mean
BGR_MEAN = np.array([91.4953, 103.8827, 131.0912], dtype=np.float32)

# Synthetic noise settings
NOISE_STD = 45.0
ADD_SHAPES = True
NUM_SHAPES_PER_IMG = (1, 4)

# --------- Preprocess (matches model pipeline) ---------
def preprocess_caffe_bgr_from_pil(pil_img, size=(224, 224)):
    rgb = np.asarray(pil_img.convert('RGB'))
    bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    bgr = cv2.resize(bgr, size, interpolation=cv2.INTER_LINEAR).astype(np.float32)
    bgr -= BGR_MEAN
    x = np.transpose(bgr, (2, 0, 1))  # CHW
    return np.expand_dims(x, 0)       # NCHW (1,3,H,W)

# --------- Synthetic image generator ---------
rng = np.random.default_rng(RNG_SEED)

def random_bgr_image(size):
    h, w = size[1], size[0]
    base = rng.normal(loc=BGR_MEAN, scale=NOISE_STD, size=(h, w, 3)).astype(np.float32)
    base = np.clip(base, 0, 255)

    if ADD_SHAPES:
        img = base.copy().astype(np.uint8)
        num_shapes = rng.integers(NUM_SHAPES_PER_IMG[0], NUM_SHAPES_PER_IMG[1] + 1)
        for _ in range(num_shapes):
            color = tuple(int(c) for c in rng.integers(0, 256, size=3))  # BGR
            thickness = int(rng.integers(1, 4))
            shape_type = int(rng.integers(0, 3))  # 0=circle, 1=rectangle, 2=line
            if shape_type == 0:
                center = (int(rng.integers(0, w)), int(rng.integers(0, h)))
                radius = int(rng.integers(min(h, w) // 20, min(h, w) // 6))
                cv2.circle(img, center, radius, color, thickness)
            elif shape_type == 1:
                p1 = (int(rng.integers(0, w)), int(rng.integers(0, h)))
                p2 = (int(rng.integers(0, w)), int(rng.integers(0, h)))
                cv2.rectangle(img, p1, p2, color, thickness)
            else:
                p1 = (int(rng.integers(0, w)), int(rng.integers(0, h)))
                p2 = (int(rng.integers(0, w)), int(rng.integers(0, h)))
                cv2.line(img, p1, p2, color, thickness)
        base = img.astype(np.float32)

    pil_img = Image.fromarray(cv2.cvtColor(base.astype(np.uint8), cv2.COLOR_BGR2RGB))
    return pil_img

# --------- Build synthetic calibration dataset ---------
sample_inputs = []
for i in range(NUM_SAMPLES):
    pil_img = random_bgr_image(IMG_SIZE)
    arr = preprocess_caffe_bgr_from_pil(pil_img, size=IMG_SIZE)
    sample_inputs.append(arr)
    if i < 8:
        print(f"[Synthetic] Sample {i}: shape={arr.shape} dtype={arr.dtype}")

# --------- Validate ---------
expected_shape = (1, 3, IMG_SIZE[1], IMG_SIZE[0])  # (N, C, H, W)
all_shapes_ok = all(arr.shape == expected_shape for arr in sample_inputs)
all_dtypes_ok = all(arr.dtype == np.float32 for arr in sample_inputs)
print("All shapes correct?:", all_shapes_ok)
print("All dtypes float32?:", all_dtypes_ok)
if not all_shapes_ok or not all_dtypes_ok or len(sample_inputs) == 0:
    raise ValueError("Calibration samples invalid: shape/dtype mismatch or empty dataset.")

# --------- Package for quantization ---------
calibration_data = {INPUT_NAME: sample_inputs}
print(f"[OK] calibration_data['{INPUT_NAME}'] has {len(calibration_data[INPUT_NAME])} samples (each {expected_shape}).")


Quantize ONNX model using AI Hub (INT8 weights & activations).
-------------------------------------------------------------

In [ ]:
# Quantize the compiled ResNet50 emotion model to INT8 using representative calibration data for efficient on-device inference.

import qai_hub as hub

print("[INFO] Submitting quantization job (INT8 weights & activations)...")
quantize_job = hub.submit_quantize_job(
    model=static_model,
    calibration_data=calibration_data,  # List of per-sample arrays (each (1,3,H,W))
    weights_dtype=hub.QuantizeDtype.INT8,
    activations_dtype=hub.QuantizeDtype.INT8,
)

quantize_job.wait()
quantized_model = quantize_job.get_target_model()
print("[INFO] Quantization complete. Model saved at:", quantized_model)

Compile quantized model to TFLite for RB3 deployment using AI Hub.
-----------------------------------------------------------------

In [ ]:
# Compile the INT8-quantized emotion model to TFLite and download it for deployment on the RB3 Gen 2 device

import qai_hub as hub

print("[INFO] Submitting TFLite compile job...")
compile_job = hub.submit_compile_job(
    model=quantized_model,
    device=hub.Device("Dragonwing RB3 Gen 2 Vision Kit"),
    name="EmotionModel_TFLite",
    input_specs=None,
    options="--target_runtime tflite",
)

compile_job.wait()
tflite_model = compile_job.get_target_model()
tflite_model.download("emotion_quant_model.tflite")

print("[INFO] TFLite model downloaded successfully for RB3 deployment.")